In [2]:
from dmrApp import create_app, db
from dmrApp.models import Dmrs, Employees, Employeeroles, Post, Restaurants, Shifts, User, Standardwages
import numpy as np
from datetime import date, datetime
from datetime import timedelta
from sqlalchemy import func
import pandas as pd
import os
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()

#make payroll report into components so that each week is captured then combined at the end

C:\tools\miniconda3\envs\dmrAppConda\lib\site-packages\flask_sqlalchemy\__init__.py:833: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [3]:
db.create_all()

In [11]:
# inputs
resId=3
dateFrom=date(2020,12,27)
dateTo=date(2021,1,9)

In [49]:
#start by getting table into pandas
shiftDf=pd.read_sql_table('shifts',db.engine)
employeerolesDf=pd.read_sql_table('employeeroles',db.engine)
restaurantsDf=pd.read_sql_table('restaurants',db.engine)

In [50]:
#get date ranges for tables
weekStartDt=dateFrom
weekEndDt=dateFrom + timedelta(days=6)
weeksDict={}
i=1
while weekEndDt< dateTo:
    weeksDict[f'week{i}']=[weekStartDt,weekEndDt]
    weekStartDt=weekEndDt + timedelta(days=1)
    weekEndDt = weekEndDt + timedelta(days=7)
    i+=1
# final week calculation
weeksDict[f'week{i}']=[weeksDict[f'week{i-1}'][0] + timedelta(days=7),weeksDict[f'week{i-1}'][1] + timedelta(days=7)]
weeksDict

{'week1': [datetime.date(2020, 12, 27), datetime.date(2021, 1, 2)],
 'week2': [datetime.date(2021, 1, 3), datetime.date(2021, 1, 9)]}

In [66]:
#build dataframe for one week's payroll if it exists
dfList=[]
# i=1
for x,y in weeksDict.items():
    df=shiftDf[(shiftDf['shiftDate']>=str(y[0])) & (shiftDf['shiftDate']<=str(y[1])) & (shiftDf['restaurantId']==resId)]
    if len(df)>0:
        df=df.groupby(['name','empId','employeeRolesId','restaurantId']).sum()
        df=df[['hoursWorked','shiftTips','shiftTipsShipgarten','wages']].copy()
        df.reset_index(inplace=True)
        df['Tips']=np.where((df['restaurantId'] >=3) & (df['restaurantId']<=6),df['shiftTipsShipgarten'],df['shiftTips'])
        df.set_index(['name','empId','employeeRolesId','restaurantId'], inplace=True)
        df1=df[['hoursWorked','Tips','wages']].copy()
        df1.columns=['Hours', 'Tips','Wages']
        dfList.append(df1)
    else:
        noShiftsColumn=f'No shifts {str(y[0])} thru {str(y[1])}'


In [57]:
if len(dfList)==2:
    df2=pd.merge(dfList[0],dfList[1],how='outer', on=['name','empId','employeeRolesId','restaurantId'], 
                 suffixes=(' Week1',' Week2')).fillna(0)
    df2['Total Hours']=df2['Hours Week1']+df2['Hours Week2']
    df2['Total Tips']=df2['Tips Week1']+df2['Tips Week2']
    df2['Total Wages']=df2['Wages Week1']+df2['Wages Week2']
else:
    df2=dfList[0]

In [58]:
#get hourly rate from employee roles
rolesDf=employeerolesDf[['id','role','wage']].copy()
rolesDf.columns=['roleId','Role','Wage']
df2.reset_index(inplace=True)
df2.set_index('employeeRolesId')
df3=pd.merge(df2,rolesDf,how='left',left_on='employeeRolesId',right_on='roleId')

In [59]:
df3

,name,empId,employeeRolesId,restaurantId,Hours,Tips,Wages,roleId,Role,Wage
0,Nick,1,1,3,1.000000,1.0,2.1300,1,bartender,2.13
1,Shayan,2,3,3,2.683333,2.0,5.7155,3,bartender,2.13


In [60]:
restaurantsDf.columns=['restaurantId','Restaurant Name']
df4=pd.merge(df3,restaurantsDf,how='left',left_on='restaurantId',right_on='restaurantId')

In [61]:
df4

,name,empId,employeeRolesId,restaurantId,Hours,Tips,Wages,roleId,Role,Wage,Restaurant Name
0,Nick,1,1,3,1.000000,1.0,2.1300,1,bartender,2.13,Tysons Biergarten
1,Shayan,2,3,3,2.683333,2.0,5.7155,3,bartender,2.13,Tysons Biergarten


In [62]:
len(df4.columns)==11

11

In [67]:

if len(df4.columns)==11:
    payrollReport=df4[['name','Restaurant Name','Role','Hours','Wages','Tips']].copy()
    payrollReport['Shift Status']=noShiftsColumn
else:
    payrollReport=df4[['name','Restaurant Name','Role','Hours Week1','Hours Week2','Total Hours','Wages Week1',
                       'Wages Week2','Total Wages','Tips Week1','Tips Week2','Total Tips']].copy()
payrollReport.rename(columns={'name':'Name'},inplace=True)

In [72]:
payrollReport

,Name,Restaurant Name,Role,Hours,Wages,Tips,No shifts
0,Nick,Tysons Biergarten,bartender,1.000000,2.1300,1.0,No shifts 2020-12-27 thru 2021-01-02
1,Shayan,Tysons Biergarten,bartender,2.683333,5.7155,2.0,No shifts 2020-12-27 thru 2021-01-02


In [71]:
reportName=f"Payroll Report {weeksDict['week1'][0].strftime('%m.%d')}-{weeksDict['week2'][1].strftime('%m.%d.%y')}.xlsx"
reportName

'Payroll Report 12.27-01.09.21.xlsx'

In [73]:
formatExcel(payrollReport,reportName,'report',3,5)

In [69]:
#return excel files formatted
def formatExcel(df,fileName,sheetName,startFloat,endFloat):
    excelObj=pd.ExcelWriter(reportName)
    df.to_excel(excelObj,sheet_name=sheetName, startrow=1,header=False, index=False)
    workbook=excelObj.book
    worksheet=excelObj.sheets[sheetName]
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'border': 0})

    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value,header_format)
        width=len(value) if len(value)>8 else 8
        worksheet.set_column(col_num,col_num,width)

    formatDecimals = workbook.add_format({'num_format': '#,##0.00'})
    worksheet.set_column(startFloat,endFloat, None, formatDecimals)
    excelObj.close()